In [1]:
import pyTigerGraph as tg
import pandas as pd
import file_operations

secret="ji08rpup1652lum47ojvpuei1ievolum"
host="https://offlabel.i.tgcloud.io"  
graphname="DRKG"
username="tigergraph" 
password="utku123"


graph = tg.TigerGraphConnection(host=host, graphname=graphname)
authToken = graph.getToken(secret=secret)
authToken = authToken[0]

conn = tg.TigerGraphConnection(host=host,graphname=graphname,username=username,password=password,apiToken=authToken)

allowed_labels = ['Hetionet::CtD::Compound:Disease','GNBR::T::Compound:Disease']

diseases_list = file_operations.open_file_return_list('diseases_names.csv')

entity_name_to_id, entity_id_to_name, relation_name_to_id, allowed_relation_ids, allowed_drug_ids,disease_ids = file_operations.entity_relations(allowed_labels,file_operations.COVID) 

allowed_drug = file_operations.allowed_drug

In [7]:
disease_ids

[9079,
 9085,
 9110,
 9124,
 9127,
 9132,
 9137,
 9139,
 9159,
 9197,
 9198,
 9201,
 9208,
 9216,
 9217,
 9219,
 9222,
 9247,
 9270,
 9286,
 9295,
 9303,
 9305,
 9309,
 9312,
 9352,
 9363,
 44581,
 9394,
 9388,
 9425,
 9430,
 9431,
 9433]

In [2]:
vertex_types = conn.getVertexTypes()
vertex_types_with_ids={}

In [3]:
for vertex in vertex_types:
    v_id = [item.get('v_id') for item in conn.getVertices(vertexType=vertex) ]
    vertex_types_with_ids[vertex] = v_id


In [4]:
list_dict_df=[]
i=0
for v in vertex_types_with_ids:
        for id in vertex_types_with_ids[v]:
            if i<5:
                for item in conn.getEdges(v,id):
                    list_dict_df.append(item)
            i+=1

In [5]:
df = pd.DataFrame(list_dict_df)

In [6]:
df

,e_type,directed,from_id,from_type,to_id,to_type,attributes
0,bioarxHumGenHumGenGeneGene,True,3359,Gene,1956,Gene,{}
1,bioarxHumGenHumGenGeneGene,True,3359,Gene,200895,Gene,{}
2,bioarxHumGenHumGenGeneGene,True,3359,Gene,21,Gene,{}
3,bioarxHumGenHumGenGeneGene,True,3359,Gene,2697,Gene,{}
4,bioarxHumGenHumGenGeneGene,True,3359,Gene,10682,Gene,{}
...,...,...,...,...,...,...,...
933,STRINGBINDINGGeneGene,True,4714,Gene,374291,Gene,{}
934,STRINGBINDINGGeneGene,True,4714,Gene,55863,Gene,{}
935,DGIDBINHIBITORGeneCompound,True,4714,Gene,CHEMBL1703,Compound,{}
936,DGIDBINHIBITORGeneCompound,True,4714,Gene,CHEMBL3545320,Compound,{}


In [32]:
entity_emb = np.load('drkg/embed/DRKG_TransE_l2_entity.npy')
rel_emb = np.load('drkg/embed/DRKG_TransE_l2_relation.npy')

allowed_drug_ids = torch.tensor(allowed_drug_ids).long()
disease_ids = torch.tensor(disease_ids).long()
allowed_relation_ids = torch.tensor(allowed_relation_ids)

allowed_drug_tensors = torch.tensor(entity_emb[allowed_drug_ids])
allowed_relation_tensors = [torch.tensor(rel_emb[rel]) for rel in allowed_relation_ids]

In [33]:
threshold= 20
def score(h, r, t):
    return fn.logsigmoid(threshold - torch.norm(h + r - t, p=2, dim=-1))

allowed_drug_scores = []
drug_ids = []
for relation_tensor in range(len(allowed_relation_tensors)):
    rel_vector = allowed_relation_tensors[relation_tensor]
    for disease_id in disease_ids:
        disease_vector = entity_emb[disease_id]
        drug_score = score(allowed_drug_tensors, rel_vector, disease_vector)
        allowed_drug_scores.append(drug_score)
        drug_ids.append(allowed_drug_ids)
scores = torch.cat(allowed_drug_scores)
drug_ids = torch.cat(drug_ids)

In [34]:
idx = torch.flip(torch.argsort(scores), dims=[0])
scores = scores[idx].numpy()
drug_ids = drug_ids[idx].numpy()
_, unique_indices = np.unique(drug_ids, return_index=True)
# top 10
topk_indices = np.sort(unique_indices)[:10]
proposed_dids = drug_ids[topk_indices]
proposed_scores = scores[topk_indices]

In [35]:
for i in range(10):
    drug = int(proposed_dids[i])
    score = proposed_scores[i]

    print("{}\t{}".format(entity_id_to_name[drug], score))

Compound::DB00811	-8.011550380615517e-05
Compound::DB00993	-0.0004376895376481116
Compound::DB00635	-0.00048744698869995773
Compound::DB01082	-0.0005631227395497262
Compound::DB01234	-0.0005747968680225313
Compound::DB00982	-0.0005910185282118618
Compound::DB00563	-0.0005936846719123423
Compound::DB00290	-0.0006365859881043434
Compound::DB01394	-0.0006528208032250404
Compound::DB01222	-0.0006566540687344968
